# artsy

In [1]:
from pymongo import MongoClient
import requests

import credentials

## setup

### database

In [2]:
client = MongoClient()
# client.drop_database('artsy')
db = client.artsy

### base url

In [3]:
BASE_URL = 'https://api.artsy.net/api'

### app

In [4]:
app_name = credentials.artsy_app_name
client_id = credentials.artsy_client_id
client_secret = credentials.artsy_client_secret

### token

In [5]:
headers = {}
payload = {'client_id': client_id, 'client_secret': client_secret}
url = BASE_URL + '/tokens/xapp_token'
response = requests.post(url, params=payload, headers=headers)
token = response.json()['token']

### auxiliar

In [6]:
def change_id_field(d):
    d['_id'] = d.pop('id')

In [7]:
def get_info(subject, relationship, obj, collection):
    print('Processing %s\' %s...' % (subject, obj))
    headers = {'X-Xapp-Token': token}
    payload = {}
#     db[collection].drop()
    total_count = db[subject].count()
    count = 0
    size = 1000
    chunks = [db[subject].find()[x:x+size] for x in range(0, total_count, size)]
    for chunk in chunks:
        ds1 = []
        for r in chunk:
            ds2 = []
            url = r['_links'][relationship]['href']
            while True:
                response = requests.get(url, params=payload, headers=headers)
                if not response.ok:
                    break
                js = response.json()
                data = js['_embedded'][obj]
                if not data:
                    break
                ids = [d['id'] for d in data]
                ds2.extend(ids)
                if '_links' in js and 'next' in js['_links']:
                    url = js['_links']['next']['href']
                else:
                    url = None
                if not url:
                    break
            ds1.append({'_id': r['_id'], 'data': ds2})
            count += 1
            if count % (total_count // 10) == 0:
                print('%d %s processed.' % (count, subject))
        db[collection].insert_many(ds1)

## artsy resources
- **Status**
- **Artists**
  - **Artworks**
  - **Similar artists**
- **Artworks**
  - **Genes**
  - **Similar artworks**
- Editions
- **Genes**
- Profiles
- **Partners**
  - **Artworks**
  - **Shows**
- **Shows**
  - **Artworks**
- Fairs
- Applications
- Users
- User Details
- Collections
- Collection Items
- Images
- Search

## status

In [8]:
headers = {}
payload = {}
url = BASE_URL + '/status'
response = requests.get(url, params=payload, headers=headers)
status = response.json()['status']
print('Status: %s' % status)

Status: good


## other resources

### artists

#### artworks

In [9]:
# creates
get_info('artists', 'artworks', 'artworks', 'artists_artworks') # subject, relationship, obj, collection

Processing artists' artworks...
4993 artists processed.
9986 artists processed.
14979 artists processed.
19972 artists processed.
24965 artists processed.
29958 artists processed.
34951 artists processed.
39944 artists processed.
44937 artists processed.
49930 artists processed.


#### similar artists

In [10]:
# is_similar_to
get_info('artists', 'similar_artists', 'artists', 'similar_artists') # subject, relationship, obj, collection

Processing artists' artists...
4993 artists processed.
9986 artists processed.
14979 artists processed.
19972 artists processed.
24965 artists processed.
29958 artists processed.
34951 artists processed.
39944 artists processed.
44937 artists processed.
49930 artists processed.


### artworks

#### genes

In [11]:
# is_described_by
get_info('artworks', 'genes', 'genes', 'artworks_genes') # subject, relationship, obj, collection

Processing artworks' genes...
2607 artworks processed.
5214 artworks processed.
7821 artworks processed.
10428 artworks processed.
13035 artworks processed.
15642 artworks processed.
18249 artworks processed.
20856 artworks processed.
23463 artworks processed.
26070 artworks processed.


#### similar artworks

In [12]:
# is_similar_to
get_info('artworks', 'similar_artworks', 'artworks', 'similar_artworks') # subject, relationship, obj, collection

Processing artworks' artworks...
2607 artworks processed.
5214 artworks processed.
7821 artworks processed.
10428 artworks processed.
13035 artworks processed.
15642 artworks processed.
18249 artworks processed.
20856 artworks processed.
23463 artworks processed.
26070 artworks processed.


### partners

#### artworks

In [13]:
# owns
get_info('partners', 'artworks', 'artworks', 'partners_artworks') # subject, relationship, obj, collection

Processing partners' artworks...
146 partners processed.
292 partners processed.
438 partners processed.
584 partners processed.
730 partners processed.
876 partners processed.
1022 partners processed.
1168 partners processed.
1314 partners processed.
1460 partners processed.


#### shows

In [14]:
# organizes
get_info('partners', 'shows', 'shows', 'partners_shows') # subject, relationship, obj, collection

Processing partners' shows...
146 partners processed.
292 partners processed.
438 partners processed.
584 partners processed.
730 partners processed.
876 partners processed.
1022 partners processed.
1168 partners processed.
1314 partners processed.
1460 partners processed.


### shows

#### artworks

In [15]:
# exhibits
get_info('shows', 'artworks', 'artworks', 'shows_artworks') # subject, relationship, obj, collection

Processing shows' artworks...
790 shows processed.
1580 shows processed.
2370 shows processed.
3160 shows processed.
3950 shows processed.
4740 shows processed.
5530 shows processed.
6320 shows processed.
7110 shows processed.
7900 shows processed.
